## Goal: Basic Vizualizations
- Note: will add more after have better idea of predictors. Right now I just want to tackle priors.
    - Then could go further in, look at conditions within a state

### Urban Rural Divide

In [1]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.set_option("display.max_rows", None, "display.max_columns", None)

import warnings
warnings.filterwarnings("ignore")

In [2]:
#read in data
All2019 = pd.read_csv("General_Clean_2019")
Accident2019 = pd.read_csv("2019accident.csv")

In [46]:
#subset
Urban_Rural_1 = Accident2019[["ST_CASE","FATALS", "CITYNAME", "RUR_URBNAME", "COUNTYNAME"]]

#convert ST_Case into a accident counter (nvm this didn't work)
Urban_Rural_1.loc[:, "ST_CASE"] = Urban_Rural_1["ST_CASE"].apply(lambda x: x - 10000)

#final
Urban_Rural_1.head()

,ST_CASE,FATALS,CITYNAME,RUR_URBNAME,COUNTYNAME
0,1,1,OPELIKA,Urban,LEE (81)
1,2,1,GADSDEN,Urban,ETOWAH (55)
2,3,1,NOT APPLICABLE,Rural,CLEBURNE (29)
3,4,1,RAINBOW CITY,Rural,ETOWAH (55)
4,5,1,NOT APPLICABLE,Urban,BALDWIN (3)


In [49]:
max_value = Urban_Rural_1["ST_CASE"].max()
print(max_value)

#so why don't our accidents add up

#this is wrong, see shape below

550121


In [54]:

Urban_Rural_1.shape

(33244, 5)

In [50]:
#count missing values by column
count_nan = len(Urban_Rural_1) - Urban_Rural_1.count()
count_nan

#no missing values

ST_CASE        0
FATALS         0
CITYNAME       0
RUR_URBNAME    0
COUNTYNAME     0
dtype: int64

In [37]:
Rural_Urban_Total_Accidents = Urban_Rural_1.groupby("RUR_URBNAME").size().reset_index(name="Total Accidents")
Rural_Urban_Total_Accidents
#something is wrong here. Should be way more accidents.

## there are more accidents (See above), and no missing values, so why?

,RUR_URBNAME,Total Accidents
0,Not Reported,7
1,Rural,14671
2,Trafficway Not in State Inventory,85
3,Unknown,56
4,Urban,18425


In [51]:
# total rural
print(Urban_Rural_1[Urban_Rural_1.RUR_URBNAME == "Urban"].shape[0])
#total urban

18425


In [58]:
#Total Traffic Accidents by Urban/Rural Plot
ax = Urban_Rural_1[['Rural_UrbName','V2']].plot(kind='bar', title ="V comp", figsize=(15, 10), legend=True, fontsize=12)
ax.set_xlabel("Hour", fontsize=12)
ax.set_ylabel("V", fontsize=12)
plt.show()

In [42]:
#Total Traffic Fatalities
Rural_Urban_Total_Fatalities = Urban_Rural_1.groupby("RUR_URBNAME").sum()
Rural_Urban_Total_Fatalities.drop("ST_CASE", axis=1, inplace=True)
Rural_Urban_Total_Fatalities.head()

,FATALS
RUR_URBNAME,
Not Reported,9
Rural,16340
Trafficway Not in State Inventory,90
Unknown,62
Urban,19595


In [25]:
#check that we have right number of fatalities
Total = Urban_Rural_1['FATALS'].sum()
print (Total)

# We're good for fatalities! This is exactly right

36096


In [5]:
#Consider Adjusting for 2019 Population, etc

In [4]:
#Fatalities by City (bar) --> nvm this isn't useful
City_Fatals = Urban_Rural_1.groupby("CITYNAME").sum()
City_Fatals.head()

,FATALS
CITYNAME,
ABBEVILLE,1
ABERDEEN,8
ABILENE,9
ABINGDON,2
ABSECON,1


In [5]:
#Fatalities by County (bar) --> nvm this isn't useful
County_Fatals = Urban_Rural_1.groupby("COUNTYNAME").sum()
County_Fatals.head()

,FATALS
COUNTYNAME,
ABBEVILLE (1),1
ACADIA (1),14
ACCOMACK (1),5
ADA (1),29
ADAIR (1),12


### By Time

###### Month

###### Day in Month

###### Day in Week

###### Time/Hour of Day

### Weather

### Road Type